In [1]:
# Self Play Experiment

In [2]:
from MCTS import MCTS
from TSP import TSPGame
from NETS import RecurrentNN, create_training_set, ConvolutionalNN, GraphConvolutionalNN
from EVALUATE import evaluation_run, plot_comparison, create_comparison
from SELFPLAY import Coach
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
class dotdict(dict):
    def __getattr__(self, name):
        return self[name]

In [4]:
num_node = 7
numAssess = 100
numEps = 300
validation_split = 0.2
numGens = 10
patience = 30

args = dotdict({
    # MCTS args
    'numMCTSSims': 100,        # Number of games moves for MCTS to simulate in self-play
    'num_node': num_node,              # Number of nodes in the graph (game)
    'cpuct': 1,
    
    # Self Play
    'numEps': numEps,
    'numAssess': numAssess,
    'numGens': numGens,
    'winThresh': 1.0           # 1 Thresh means replace on any total improvement

    })

rnn_args = dotdict({
    # State representation
    'history': True,           # Whether state representation should be a history of states - used for lstm
    'history_length': None,     # If None full history
    
    # NN args
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 20,
    'batch_size': 64,
    'validation_split': validation_split,
    'patience': patience
    })

cnn_args = dotdict({
    # State representation
    'history': False,           # Whether state representation should be a history of states - used for lstm
    
    # NN args
    'lr': 0.001,
    'dropout': 0.3,
    'epochs': 100,
    'batch_size': 64,
    'num_channels': 512,
    'validation_split': validation_split,
    'patience': patience
    })

gnn_args = dotdict({
    # State representation
    'history': False,           # Whether state representation should be a history of states - used for lstm
    
    # NN args
    'lr': 0.001,
    'epochs': 100,
    'batch_size': 64,
    'validation_split': validation_split,
    'patience': patience
    })

gnn_args.update(args)
cnn_args.update(args)
rnn_args.update(args)

In [5]:
# Create RNN
rnn = RecurrentNN(rnn_args)
gnn = GraphConvolutionalNN(gnn_args)
cnn = ConvolutionalNN(cnn_args)

In [6]:
# Self-Train

In [9]:
c = Coach(gnn, gnn_args)

In [10]:
%%time
c.arena()

Assessing old...


KeyboardInterrupt: 

In [23]:
# Save
c.nnet.save_model('GNN_SelfPlay')

Loaded model from disk
